In [12]:
using BenchmarkTools
using Random

n = 6*10^4
Typ = UInt64
dict = Dict{Typ, Typ}()
for i in 1:n
    dict[i] = i
end

existing_keys = collect(keys(dict))
shuffle!(existing_keys)
test_keys = existing_keys[1:1000]

bench = @benchmark for key in test_keys 
    dict[key]
end
display(bench)


BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  166.300 μs …  51.076 ms  ┊ GC (min … max): 0.00% … 99.41%
 Time  (median):     196.800 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   230.125 μs ± 641.943 μs  ┊ GC (mean ± σ):  5.30% ±  2.20%

  ██▆▅▄▅▇▆▅▄▃▂▂▁▁                                               ▂
  █████████████████████▇▇▇▇▇▇▇▇▆▆▄▆▅▅▅▅▅▆▅▅▄▄▅▄▅▆▃▅▃▅▅▄▃▂▄▃▄▄▄▅ █
  166 μs        Histogram: log(frequency) by time        662 μs <

 Memory estimate: 69.44 KiB, allocs estimate: 3444.

Look up for big dict is approx >>200 ns, for UInt16, UInt32,
smaller dicts are approx 100 ns. And changing the value of the dict is approx 300ns.

In [60]:
println(1/ (100* 10^(-9)))
println(1/ (200* 10^(-9)))
println(1/ (300* 10^(-9)))

1.0e7
5.0e6
3.333333333333333e6


This means I can do max 3e6~10e6 lookups or changes /s.

In [58]:
n = 10^5
Typ = UInt32
dict = Dict{Typ, Typ}()
for i in 1:n
    dict[i] = i
end

existing_keys = collect(keys(dict))
shuffle!(existing_keys)
test_keys = [existing_keys[rand(1:length(existing_keys))] for _ in 1:100]

println(sizeof(dict))

64


In [59]:
bench = @benchmark for key in test_keys 
    dict[key]
end
display(bench)

bench2 = @benchmark for key in test_keys 
    dict[key] +=10
end
display(bench2)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  16.200 μs …  2.361 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.900 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   19.703 μs ± 43.230 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▆▂▁  ▁                                                     ▁
  █████████▇▆▅▆▅▅▅▅▄▅▅▅▆▅▃▄▅▅▅▄▅▅▅▅▅▃▁▄▅▃▄▄▄▄▄▁▄▄▄▅▅▄▄▃▄▃▃▅▃▅ █
  16.2 μs      Histogram: log(frequency) by time      65.3 μs <

 Memory estimate: 6.25 KiB, allocs estimate: 300.

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  24.700 μs … 820.000 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     26.800 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   30.372 μs ±  22.535 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▆▅▄▄▃▃▄▂▁▁   ▁▁                                            ▂
  ██████████████▇██▇▆▆▇▆▇▆▆▆▅▅▅▆▅▅▄▄▃▄▄▆▆▄▅▅▄▅▄▅▅▅▄▄▄▃▁▁▃▄▁▃▃▃ █
  24.7 μs       Histogram: log(frequency) by time      95.4 μs <

 Memory estimate: 7.81 KiB, allocs estimate: 400.

List used as dict are faster

In [57]:
n = 10^4
Typ = UInt32

# Initialize the array with zeros
list = zeros(Typ, n)

# Assign values from 1 to 100
for i in 1:n
    list[i] = i
end

existing_keys = collect(1:n)
shuffle!(existing_keys)
test_keys = [existing_keys[rand(1:length(existing_keys))] for _ in 1:100]
println("")



  0.000006 seconds


10000-element Vector{UInt32}:
 0x00000001
 0x00000002
 0x00000003
 0x00000004
 0x00000005
 0x00000006
 0x00000007
 0x00000008
 0x00000009
 0x0000000a
          ⋮
 0x00002708
 0x00002709
 0x0000270a
 0x0000270b
 0x0000270c
 0x0000270d
 0x0000270e
 0x0000270f
 0x00002710

In [56]:
bench = @benchmark for key in test_keys 
    list[key]
end
display(bench)

bench2 = @benchmark for key in test_keys 
    list[key] +=1
end
display(bench2)

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  12.300 μs … 685.900 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.100 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.691 μs ±  16.971 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆██▆▄▁                  ▁▂▁                                  ▂
  ███████▅▅▆▆▇█▇▅▆▆▆▆▅▅▄▆████▇▇▅▆▅▄▄▅▄▅▄▃▄▄▄▄▁▄▃▅▇▇▆▆▅▄▁▄▄▃▃▄▄ █
  12.3 μs       Histogram: log(frequency) by time      34.6 μs <

 Memory estimate: 5.89 KiB, allocs estimate: 277.

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  21.500 μs … 956.600 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     22.800 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   25.710 μs ±  22.184 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▆▃▃▂▂▁ ▃▃                                                  ▂
  ████████████▇▇▇▇▅▇█▇▆▇▆▆▅▄▅▅▄▄▁▄▅▄▄▅▄▁▄▃▃▅▅▄▄▅▃▄▄▅▅▁▄▃▄▄▃▄▄▄ █
  21.5 μs       Histogram: log(frequency) by time      80.6 μs <

 Memory estimate: 7.67 KiB, allocs estimate: 391.